In [0]:
%sql
-- %fs
-- ls dbfs:/user/hive/warehouse/

In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/euro

res0: Boolean = true

In [0]:
%sql
CREATE or replace TABLE euro (
    team_name VARCHAR(20),
    grp VARCHAR(1)
  
);
 
-- Insert initial data for Euro Cup teams
INSERT INTO euro (team_name, grp)
VALUES
    ('Germany', 'A'), ('Scotland', 'A'), ('Hungary', 'A'), ('Switzerland', 'A'),
    ('Spain', 'B'), ('Croatia', 'B'), ('Italy', 'B'), ('Albania', 'B'),
    ('Slovenia', 'C'), ('Denmark', 'C'), ('Serbia', 'C'), ('England', 'C'),
    ('Netherlands', 'D'), ('Austria', 'D'), ('France', 'D'),('Poland', 'D'),
    ('Belgium', 'E'),('Slovakia', 'E'), ('Romania', 'E'),('Finland', 'E'),
    ('Turkey', 'E'),('Portugal', 'F'), ('Czechia', 'F'),('Israel', 'F');
 

num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql

select * from euro

team_name,grp
Germany,A
Scotland,A
Hungary,A
Switzerland,A
Spain,B
Croatia,B
Italy,B
Albania,B
Slovenia,C
Denmark,C


In [0]:
%sql
SELECT 
    CONCAT(t1.team_name, ' X ', t2.team_name) AS match,
    t1.team_name as team1,
    t2.team_name as team2,
    t1.grp,
    FLOOR(RAND() * 5) AS team1_goals,
    FLOOR(RAND() * 5) AS team2_goals,
    CASE
            WHEN team1_goals > team2_goals THEN 3
            WHEN team1_goals = team2_goals THEN 1 
            ELSE 0
        END AS team1_points,
        CASE
            WHEN team2_goals > team1_goals THEN 3
            WHEN team2_goals = team1_goals THEN 1
            ELSE 0
        END AS team2_points
    FROM 
        euro t1
CROSS JOIN 
    euro t2 
WHERE 
    t1.grp = t2.grp 
    AND t1.team_name > t2.team_name;


match,team1,team2,grp,team1_goals,team2_goals,team1_points,team2_points
Scotland X Hungary,Scotland,Hungary,A,3,4,0,3
Scotland X Germany,Scotland,Germany,A,2,0,3,0
Hungary X Germany,Hungary,Germany,A,2,3,0,3
Switzerland X Hungary,Switzerland,Hungary,A,3,1,3,0
Switzerland X Scotland,Switzerland,Scotland,A,2,2,1,1
Switzerland X Germany,Switzerland,Germany,A,3,3,1,1
Spain X Albania,Spain,Albania,B,2,3,0,3
Spain X Italy,Spain,Italy,B,1,2,0,3
Spain X Croatia,Spain,Croatia,B,3,4,0,3
Croatia X Albania,Croatia,Albania,B,3,0,3,0


In [0]:
%sql
WITH cte1 AS (
    SELECT 
        CONCAT(t1.team_name, ' X ', t2.team_name) AS match,
        t1.team_name AS team1,
        t2.team_name AS team2,
        t1.grp,
        FLOOR(RAND() * 5) AS team1_goals,
        FLOOR(RAND() * 5) AS team2_goals,
        CASE
            WHEN team1_goals > team2_goals THEN 3
            WHEN team1_goals = team2_goals THEN 1 
            ELSE 0
        END AS team1_points,
        CASE
            WHEN team2_goals > team1_goals THEN 3
            WHEN team2_goals = team1_goals THEN 1
            ELSE 0
        END AS team2_points
    FROM 
        euro t1
    CROSS JOIN 
        euro t2 
    WHERE 
        t1.grp = t2.grp 
        AND t1.team_name > t2.team_name
),
cte2 AS (
    SELECT 
        team,
        grp,
        SUM(goals_forward) AS goals_forward,
        SUM(goals_against) AS goals_against,
        SUM(team1_points) AS pts 
    FROM (
        SELECT  
            team1 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team1_points 
        FROM cte1
        UNION ALL
        SELECT  
            team2 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team2_points 
        FROM cte1
    ) AS temp
    GROUP BY team, grp
)
SELECT 
    team, 
    grp,
    goals_forward,
    goals_against,
    (goals_forward - goals_against) AS gd,
    pts,
    FLOOR(pts / 3) AS wins,
    FLOOR(pts % 3) AS draws,
    3 - (FLOOR(pts / 3) + FLOOR(pts % 3)) AS lose
FROM 
    cte2 ORDER BY grp,pts DESC;


team,grp,goals_forward,goals_against,gd,pts,wins,draws,lose
Scotland,A,8,10,-2,6,2,0,1
Hungary,A,6,3,3,6,2,0,1
Switzerland,A,7,6,1,6,2,0,1
Germany,A,9,5,4,0,0,0,3
Croatia,B,6,6,0,7,2,1,0
Spain,B,9,3,6,7,2,1,0
Italy,B,8,4,4,3,1,0,2
Albania,B,11,7,4,0,0,0,3
Slovenia,C,11,3,8,7,2,1,0
Denmark,C,7,7,0,6,2,0,1


In [0]:
%sql
WITH cte1 AS (
    SELECT 
        CONCAT(t1.team_name, ' X ', t2.team_name) AS match,
        t1.team_name AS team1,
        t2.team_name AS team2,
        t1.grp,
        FLOOR(RAND() * 5) AS team1_goals,
        FLOOR(RAND() * 5) AS team2_goals,
        CASE
            WHEN team1_goals > team2_goals THEN 3
            WHEN team1_goals = team2_goals THEN 1 
            ELSE 0
        END AS team1_points,
        CASE
            WHEN team2_goals > team1_goals THEN 3
            WHEN team2_goals = team1_goals THEN 1
            ELSE 0
        END AS team2_points
    FROM 
        euro t1
    CROSS JOIN 
        euro t2 
    WHERE 
        t1.grp = t2.grp 
        AND t1.team_name > t2.team_name
),
cte2 AS (
    SELECT 
        team,
        grp,
        SUM(goals_forward) AS goals_forward,
        SUM(goals_against) AS goals_against,
        SUM(team1_points) AS pts 
    FROM (
        SELECT  
            team1 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team1_points 
        FROM cte1
        UNION ALL
        SELECT  
            team2 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team2_points 
        FROM cte1
    ) AS temp
    GROUP BY team, grp
)
SELECT 
    team, 
    grp,
    goals_forward,
    goals_against,
    (goals_forward - goals_against) AS gd,
    pts,
    FLOOR(pts / 3) AS wins,
    FLOOR(pts % 3) AS draws,
    3 - (FLOOR(pts / 3) + FLOOR(pts % 3)) AS lose,
    row_number() OVER(PARTITION BY GRP ORDER BY pts desc) AS  rank
FROM 
    cte2 ORDER BY grp,pts DESC;


team,grp,goals_forward,goals_against,gd,pts,wins,draws,lose,rank
Scotland,A,4,5,-1,6,2,0,1,1
Hungary,A,7,3,4,6,2,0,1,2
Germany,A,3,1,2,4,1,1,1,3
Switzerland,A,4,7,-3,1,0,1,2,4
Italy,B,7,3,4,7,2,1,0,1
Croatia,B,4,3,1,4,1,1,1,2
Albania,B,4,3,1,4,1,1,1,3
Spain,B,3,5,-2,1,0,1,2,4
Serbia,C,10,5,5,6,2,0,1,1
England,C,10,4,6,4,1,1,1,2


#Round of 16

In [0]:
%sql
with cte3 as(
  WITH cte1 AS (
    SELECT 
        CONCAT(t1.team_name, ' X ', t2.team_name) AS match,
        t1.team_name AS team1,
        t2.team_name AS team2,
        t1.grp,
        FLOOR(RAND() * 5) AS team1_goals,
        FLOOR(RAND() * 5) AS team2_goals,
        CASE
            WHEN team1_goals > team2_goals THEN 3
            WHEN team1_goals = team2_goals THEN 1 
            ELSE 0
        END AS team1_points,
        CASE
            WHEN team2_goals > team1_goals THEN 3
            WHEN team2_goals = team1_goals THEN 1
            ELSE 0
        END AS team2_points
    FROM 
        euro t1
    CROSS JOIN 
        euro t2 
    WHERE 
        t1.grp = t2.grp 
        AND t1.team_name > t2.team_name
),
cte2 AS (
    SELECT 
        team,
        grp,
        SUM(goals_forward) AS goals_forward,
        SUM(goals_against) AS goals_against,
        SUM(team1_points) AS pts 
    FROM (
        SELECT  
            team1 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team1_points 
        FROM cte1
        UNION ALL
        SELECT  
            team2 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team2_points 
        FROM cte1
    ) AS temp
    GROUP BY team, grp
)
SELECT 
    team, 
    grp,
    goals_forward,
    goals_against,
    (goals_forward - goals_against) AS gd,
    pts,
    FLOOR(pts / 3) AS wins,
    FLOOR(pts % 3) AS draws,
    3 - (FLOOR(pts / 3) + FLOOR(pts % 3)) AS lose,
    row_number() OVER(PARTITION BY GRP ORDER BY pts desc,(goals_forward - goals_against) asc) AS  rank
FROM 
    cte2 ORDER BY grp,pts DESC

)

(select * from cte3 where rank <3) 
union
(select * from cte3 where rank = 3) order by rank,pts desc,gd desc,goals_forward desc limit 16



team,grp,goals_forward,goals_against,gd,pts,wins,draws,lose,rank
Poland,D,11,2,9,9,3,0,0,1
Slovakia,E,7,4,3,9,3,0,0,1
Spain,B,9,5,4,6,2,0,1,1
Slovenia,C,9,6,3,6,2,0,1,1
Germany,A,4,10,-6,6,2,0,1,1
Israel,F,4,8,-4,3,1,0,2,1
Romania,E,12,6,6,7,2,1,0,2
Serbia,C,10,2,8,6,2,0,1,2
Croatia,B,8,3,5,6,2,0,1,2
Switzerland,A,5,6,-1,6,2,0,1,2


In [0]:
%sql
create or replace table group_16 as ((with cte3 as(
  WITH cte1 AS (
    SELECT 
        CONCAT(t1.team_name, ' X ', t2.team_name) AS match,
        t1.team_name AS team1,
        t2.team_name AS team2,
        t1.grp,
        FLOOR(RAND() * 5) AS team1_goals,
        FLOOR(RAND() * 5) AS team2_goals,
        CASE
            WHEN team1_goals > team2_goals THEN 3
            WHEN team1_goals = team2_goals THEN 1 
            ELSE 0
        END AS team1_points,
        CASE
            WHEN team2_goals > team1_goals THEN 3
            WHEN team2_goals = team1_goals THEN 1
            ELSE 0
        END AS team2_points
    FROM 
        euro t1
    CROSS JOIN 
        euro t2 
    WHERE 
        t1.grp = t2.grp 
        AND t1.team_name > t2.team_name
),
cte2 AS (
    SELECT 
        team,
        grp,
        SUM(goals_forward) AS goals_forward,
        SUM(goals_against) AS goals_against,
        SUM(team1_points) AS pts 
    FROM (
        SELECT  
            team1 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team1_points 
        FROM cte1
        UNION ALL
        SELECT  
            team2 AS team,
            grp,
            team1_goals AS goals_forward,
            team2_goals AS goals_against,
            team2_points 
        FROM cte1
    ) AS temp
    GROUP BY team, grp
)
SELECT 
    team, 
    grp,
    goals_forward,
    goals_against,
    (goals_forward - goals_against) AS gd,
    pts,
    FLOOR(pts / 3) AS wins,
    FLOOR(pts % 3) AS draws,
    3 - (FLOOR(pts / 3) + FLOOR(pts % 3)) AS lose,
    row_number() OVER(PARTITION BY GRP ORDER BY pts desc,(goals_forward - goals_against) asc) AS  rank
FROM 
    cte2 ORDER BY grp,pts DESC

)

select *,concat(grp,rank) as grp_wise_rank from ((select * from cte3 where rank <3) 
union
(select * from cte3 where rank = 3) order by rank,pts desc,gd desc,goals_forward desc limit 16) order by rank,grp))




num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from group_16 order by rank,grp;

team,grp,goals_forward,goals_against,gd,pts,wins,draws,lose,rank,grp_wise_rank
Scotland,A,7,5,2,7,2,1,0,1,A1
Albania,B,5,9,-4,7,2,1,0,1,B1
England,C,3,5,-2,7,2,1,0,1,C1
Austria,D,8,9,-1,6,2,0,1,1,D1
Belgium,E,5,14,-9,9,3,0,0,1,E1
Czechia,F,4,7,-3,6,2,0,1,1,F1
Switzerland,A,9,6,3,7,2,1,0,2,A2
Spain,B,9,6,3,6,2,0,1,2,B2
Slovenia,C,12,6,6,7,2,1,0,2,C2
Poland,D,7,5,2,6,2,0,1,2,D2


In [0]:
%fs
rm -r dbfs:/user/hive/warehouse/eurocupfixture

res1: Boolean = true

In [0]:
%sql
CREATE TABLE EuroCupFixture (
    Qteams VARCHAR(255),
    B1 VARCHAR(255),
    C1 VARCHAR(255),
    E1 VARCHAR(255),
    F1 VARCHAR(255)
);

INSERT INTO EuroCupFixture (QTeams,B1, C1,E1, F1) VALUES
('ABCD', 'A3', 'D3', 'B3', 'C3'),
('ABCE', 'A3', 'E3', 'B3', 'C3'),
('ABCF', 'A3', 'F3', 'B3', 'C3'),
('ABDE', 'D3', 'E3', 'A3', 'B3'),
('ABDF', 'D3', 'F3', 'A3', 'B3'),
('ABEF', 'E3', 'F3', 'B3', 'A3'),
('ACDE', 'E3', 'D3', 'C3', 'A3'),
('ACDF', 'F3', 'D3', 'C3', 'A3'),
('ACEF', 'E3', 'F3', 'C3', 'A3'),
('ADEF', 'E3', 'F3', 'D3', 'A3'),
('BCDE', 'E3', 'D3', 'B3', 'C3'),
('BCDF', 'F3', 'D3', 'C3', 'B3'),
('BCEF', 'F3', 'E3', 'C3', 'B3'),
('BDEF', 'F3', 'E3', 'D3', 'B3'),
('CDEF', 'F3', 'E3', 'D3', 'C3');


num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
select * from eurocupfixture

Qteams,B1,C1,E1,F1
ABCD,A3,D3,B3,C3
ABCE,A3,E3,B3,C3
ABCF,A3,F3,B3,C3
ABDE,D3,E3,A3,B3
ABDF,D3,F3,A3,B3
ABEF,E3,F3,B3,A3
ACDE,E3,D3,C3,A3
ACDF,F3,D3,C3,A3
ACEF,E3,F3,C3,A3
ADEF,E3,F3,D3,A3


In [0]:
%sql
create or replace view  third_grp as (SELECT concat_ws('',collect_list(grp)) as third_grp FROM group_16 WHERE rank = 3);


In [0]:
%sql
select * from third_grp

third_grp
BDEF


In [0]:
%sql
create or replace view third_group_fix as (select * from eurocupfixture where Qteams=(select * from third_grp))

In [0]:
%sql
select * from third_group_fix

Qteams,B1,C1,E1,F1
BDEF,F3,E3,D3,B3


In [0]:
%sql
with cte1 as (select team,grp_wise_rank from group_16 where grp_wise_rank = "B1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select B1 from third_group_fix)
union all 
select team,grp_wise_rank from group_16 where grp_wise_rank = "A1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select F1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select E1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select C1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "A2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "B2"


)select * from cte1

team,grp_wise_rank
Albania,B1
Portugal,F3
Scotland,A1
Slovenia,C2
Czechia,F1
Croatia,B3
Poland,D2
Romania,E2
Belgium,E1
Austria,D3


In [0]:
%sql
with cte1 as (select team,grp_wise_rank from group_16 where grp_wise_rank = "B1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select B1 from third_group_fix)
union all 
select team,grp_wise_rank from group_16 where grp_wise_rank = "A1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select F1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select E1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select C1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "A2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "B2"


)select *,ntile(8) over(order by (select 0)) as match_no from cte1;

team,grp_wise_rank,match_no
Albania,B1,1
Portugal,F3,1
Scotland,A1,2
Slovenia,C2,2
Czechia,F1,3
Croatia,B3,3
Poland,D2,4
Romania,E2,4
Belgium,E1,5
Austria,D3,5


In [0]:
%sql
with cte2 as (with cte1 as (select team,grp_wise_rank from group_16 where grp_wise_rank = "B1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select B1 from third_group_fix)
union all 
select team,grp_wise_rank from group_16 where grp_wise_rank = "A1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select F1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select E1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select C1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "A2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "B2"


)select *,ntile(8) over(order by (select 0)) as match_no from cte1)

select *,rand() from cte2


team,grp_wise_rank,match_no,rand()
Albania,B1,1,0.8982498254074912
Portugal,F3,1,0.47652525469796336
Scotland,A1,2,0.8699937014042743
Slovenia,C2,2,0.42551473677507823
Czechia,F1,3,0.09067148031781003
Croatia,B3,3,0.4386617845529325
Poland,D2,4,0.611732982884625
Romania,E2,4,0.8427763842415688
Belgium,E1,5,0.39390781493284444
Austria,D3,5,0.4422778152011475


In [0]:
%sql
with cte2 as (with cte1 as (select team,grp_wise_rank from group_16 where grp_wise_rank = "B1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select B1 from third_group_fix)
union all 
select team,grp_wise_rank from group_16 where grp_wise_rank = "A1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select F1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select E1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select C1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "A2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "B2"


)select *,ntile(8) over(order by (select 0)) as match_no,rand() as rand  from cte1)

SELECT 
    *,
    LEAD(rand, 1, LAG(rand, 1) OVER (PARTITION BY match_no ORDER BY match_no)) 
        OVER (PARTITION BY match_no ORDER BY match_no) AS next_rand
FROM 
    cte2;



team,grp_wise_rank,match_no,rand,next_rand
Albania,B1,1,0.9458654021168325,0.7992570063564063
Portugal,F3,1,0.7992570063564063,0.9458654021168325
Scotland,A1,2,0.8418645243518852,0.8388043427755678
Slovenia,C2,2,0.8388043427755678,0.8418645243518852
Czechia,F1,3,0.5993937004192601,0.8875693633080284
Croatia,B3,3,0.8875693633080284,0.5993937004192601
Poland,D2,4,0.05951478403041177,0.3895613576649405
Romania,E2,4,0.3895613576649405,0.05951478403041177
Belgium,E1,5,0.02389260179491992,0.21305594265362415
Austria,D3,5,0.21305594265362415,0.02389260179491992


In [0]:
%sql
create or replace table round16_result as(
with cte3 as(
with cte2 as (with cte1 as (select team,grp_wise_rank from group_16 where grp_wise_rank = "B1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select B1 from third_group_fix)
union all 
select team,grp_wise_rank from group_16 where grp_wise_rank = "A1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select F1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "E1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select E1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "D1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "F2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "C1"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = (select C1 from third_group_fix)
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "A2"
union all
select team,grp_wise_rank from group_16 where grp_wise_rank = "B2"


)select *,ntile(8) over(order by (select 0)) as match_no,rand() as rand  from cte1)

SELECT 
    *,
    LEAD(rand, 1, LAG(rand, 1) OVER (PARTITION BY match_no ORDER BY match_no)) 
        OVER (PARTITION BY match_no ORDER BY match_no) AS next_rand
FROM 
    cte2

) select *, 
case
    when rand > next_rand then "win"
    when rand < next_rand then "lose"
end as result from cte3)



num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from round16_result;


team,grp_wise_rank,match_no,rand,next_rand,result
Albania,B1,1,0.9401903009144282,0.7782476004464558,win
Portugal,F3,1,0.7782476004464558,0.9401903009144282,lose
Scotland,A1,2,0.6115132543428442,0.105157782631349,win
Slovenia,C2,2,0.105157782631349,0.6115132543428442,lose
Czechia,F1,3,0.700889231154397,0.006898908987940655,win
Croatia,B3,3,0.006898908987940655,0.700889231154397,lose
Poland,D2,4,0.3188960795742851,0.8125518464763534,lose
Romania,E2,4,0.8125518464763534,0.3188960795742851,win
Belgium,E1,5,0.1068345497768215,0.6007611383943818,lose
Austria,D3,5,0.6007611383943818,0.1068345497768215,win


In [0]:
%sql
with quarter_final_results as (with quarter_finals as( select team,grp_wise_rank,ntile(4) over(order by (select 0)) as match_no,rand() as rand   from round16_result where result="win")
select * from quarter_finals)
select * , 
case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_final_results

team,grp_wise_rank,match_no,rand,result
Albania,B1,1,0.502697024509794,win
Scotland,A1,1,0.48892698708131677,lose
Czechia,F1,2,0.5500775541073525,lose
Romania,E2,2,0.6937941500008531,win
Austria,D3,3,0.13697784819708592,lose
Portugal,F2,3,0.39234598009803767,win
England,C1,4,0.1647332290759741,lose
Spain,B2,4,0.7117782221057908,win


In [0]:
%sql
with semi_finals as (with quarter_final_results as (with quarter_finals as( select team,grp_wise_rank,ntile(4) over(order by (select 0)) as match_no,rand() as rand   from round16_result where result="win")
select * from quarter_finals)
select * , 
case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_final_results)


select team,grp_wise_rank, ntile(4) over(order by (select 0)) as match_no,rand() as rand from semi_finals where result="win"


team,grp_wise_rank,match_no,rand
Scotland,A1,1,0.687893120261882
Romania,E2,2,0.8733079206209842
Portugal,F2,3,0.1421749482259671
England,C1,4,0.28780498659885456


In [0]:
%sql
with quarter_finals_result as (with semi_finals as (with quarter_final_results as (with quarter_finals as( select team,grp_wise_rank,ntile(4) over(order by (select 0)) as match_no,rand() as rand   from round16_result where result="win")
select * from quarter_finals)
select * , 
case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_final_results)


select team,grp_wise_rank, ntile(2) over(order by (select 0)) as match_no,rand() as rand from semi_finals where result="win")

select *,case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_finals_result

team,grp_wise_rank,match_no,rand,result
Scotland,A1,1,0.6723347852046977,win
Romania,E2,1,0.1513441105683322,lose
Austria,D3,2,0.8700336104462315,win
England,C1,2,0.5986344001166054,lose


In [0]:
%sql
with finals as (
  with quarter_finals_result as (with semi_finals as (with quarter_final_results as (with quarter_finals as( select team,grp_wise_rank,ntile(4) over(order by (select 0)) as match_no,rand() as rand   from round16_result where result="win")
select * from quarter_finals)
select * , 
case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_final_results)


select team,grp_wise_rank, ntile(2) over(order by (select 0)) as match_no,rand() as rand from semi_finals where result="win")

select *,case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_finals_result
)

select team from finals  where result = "win";

team
Albania
Spain


In [0]:
%sql
with winner as (with finals as (
  with quarter_finals_result as (with semi_finals as (with quarter_final_results as (with quarter_finals as( select team,grp_wise_rank,ntile(4) over(order by (select 0)) as match_no,rand() as rand   from round16_result where result="win")
select * from quarter_finals)
select * , 
case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_final_results)


select team,grp_wise_rank, ntile(2) over(order by (select 0)) as match_no,rand() as rand from semi_finals where result="win")

select *,case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_finals_result
)

select team,rand() as score,ntile(1) over(order by (select 0))as match_no from finals  where result = "win")

select *,
case
when score>lead(score,1,lag(score,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from winner;

team,score,match_no,result
Czechia,0.11916448543355462,1,lose
Austria,0.5527283831148252,1,win


In [0]:
%sql
with finalist as (with winner as (with finals as (
  with quarter_finals_result as (with semi_finals as (with quarter_final_results as (with quarter_finals as( select team,grp_wise_rank,ntile(4) over(order by (select 0)) as match_no,rand() as rand   from round16_result where result="win")
select * from quarter_finals)
select * , 
case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_final_results)


select team,grp_wise_rank, ntile(2) over(order by (select 0)) as match_no,rand() as rand from semi_finals where result="win")

select *,case
when rand>lead(rand,1,lag(rand,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from quarter_finals_result
)

select team,rand() as score,ntile(1) over(order by (select 0))as match_no from finals  where result = "win")

select *,
case
when score>lead(score,1,lag(score,1) over(partition by match_no order by match_no) ) over(partition by match_no order by match_no) then "win"
else "lose"
end as result from winner)
select team as WINNER from finalist where result="win"

WINNER
Scotland
